In [5]:
import run_simulation_eb as sim
import pygem_eb.massbalance as mb
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [6]:
# initialize the model
args = sim.get_args()
gn = args.glac_no[0]
climate = sim.initialize_model(gn,args)
massbal = mb.massBalance(0,args,climate)

1 glaciers in region 1 are included in this model run: ['00570']
This study is focusing on 1 glaciers in region [1]
19 layers initialized for bin 0


In [7]:
%lprun -f massbal.main massbal.main()

MONTH COMPLETED: April 2023 with +0.04 and -0.01 m w.e.
Currently -6.38 C with 4 W m-2 melt energy
----------surface albedo: 0.780 -----------
-----------surface temp: -8.50 C-----------
|       snow depth: 1.85 m      8 layers      |
|       ice depth: 200.00 m      11 layers      |
--------------------layer 0---------------------
     T = -2.0 C                 h = 0.019 m 
                 p = 166 kg/m3
Water Mass : 0.00 kg/m2   Dry Mass : 3.22 kg/m2
--------------------layer 1---------------------
     T = 0.0 C                 h = 0.032 m 
                 p = 153 kg/m3
Water Mass : 0.94 kg/m2   Dry Mass : 4.96 kg/m2
MONTH COMPLETED: May 2023 with +0.13 and -0.20 m w.e.
Currently -2.12 C with 26 W m-2 melt energy
----------surface albedo: 0.770 -----------
-----------surface temp: -8.50 C-----------
|       snow depth: 1.58 m      8 layers      |
|       ice depth: 200.00 m      11 layers      |
--------------------layer 0---------------------
     T = -0.0 C                 h = 0

Timer unit: 1e-09 s

Total time: 79.0849 s
File: /home/claire/research/PyGEM-EB/pygem_eb/massbalance.py
Function: main at line 48

Line #      Hits         Time  Per Hit   % Time  Line Contents
    48                                               def main(self):
    49                                                   """
    50                                                   Runs the time loop and mass balance scheme to solve for melt, refreeze, 
    51                                                   accumulation and runoff.
    52                                                   """
    53                                                   # Get classes and time
    54         1       2015.0   2015.0      0.0          layers = self.layers
    55         1        758.0    758.0      0.0          surface = self.surface
    56         1        619.0    619.0      0.0          dt = self.dt
    57                                           
    58                                       

In [1]:
import sys,os
sys.path.append('/home/claire/research/PyGEM-EB/')
import argparse
import time
# External libraries
import numpy as np
import xarray as xr
import pandas as pd
from multiprocessing import Pool
# Internal libraries
import pygem_eb.input as eb_prms
import pygem.class_climate as class_climate
import pygem_eb.massbalance as mb
import pygem.pygem_modelsetup as modelsetup
import pygem_eb.utils as utilities

Running 1 bin(s) at [1542.0] m a.s.l. for 4.0 months starting in April, 2023


ModuleNotFoundError: No module named 'pygem_eb.utils'

In [8]:
utils = utilities.Utils()
glac_no = ['01.00570']
startdate = pd.to_datetime('2023-04-21 00:00')
enddate = pd.to_datetime('2023-08-09 00:00')
n_bins = 1
climate_input = 'AWS'

# ===== GLACIER AND TIME PERIOD SETUP =====
glacier_table = modelsetup.selectglaciersrgitable(glac_no,
                rgi_regionsO1=eb_prms.rgi_regionsO1, rgi_regionsO2=eb_prms.rgi_regionsO2,
                rgi_glac_number=eb_prms.rgi_glac_number, include_landterm=eb_prms.include_landterm,
                include_laketerm=eb_prms.include_laketerm, include_tidewater=eb_prms.include_tidewater)
dates_table = modelsetup.datesmodelrun(startyear=startdate, endyear=enddate)

gcm = class_climate.GCM(name=eb_prms.ref_gcm_name)
nans = np.empty(len(dates_table))*np.nan
if climate_input in ['GCM']:
    # ===== LOAD CLIMATE DATA =====
    tp_data, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.prec_fn, gcm.prec_vn, glacier_table,dates_table)
    temp_data, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.temp_fn, gcm.temp_vn, glacier_table,dates_table)
    dtemp_data, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.dtemp_fn, gcm.dtemp_vn, glacier_table,dates_table)
    sp_data, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.sp_fn, gcm.sp_vn, glacier_table,dates_table)
    tcc, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.tcc_fn, gcm.tcc_vn, glacier_table,dates_table)
    SWin, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.SWin_fn, gcm.SWin_vn, glacier_table,dates_table) 
    LWin, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.SWin_fn, gcm.SWin_vn, glacier_table,dates_table) 
    uwind, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.uwind_fn, gcm.uwind_vn, glacier_table,dates_table)                                                      
    vwind, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.vwind_fn, gcm.vwind_vn, glacier_table,dates_table)
    try:
        depBC, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.depBC_fn, gcm.depBC_vn, glacier_table,dates_table)
        depdust, data_hours = gcm.importGCMvarnearestneighbor_xarray(gcm.depdust_fn, gcm.depdust_vn, glacier_table,dates_table)
    except:
        depBC = nans.copy()
        depdust = nans.copy()
    elev_data = gcm.importGCMfxnearestneighbor_xarray(gcm.elev_fn, gcm.elev_vn, glacier_table)
    wind = np.sqrt(np.power(uwind[0],2)+np.power(vwind[0],2))
    winddir = np.arctan2(-uwind[0],-vwind[0]) * 180 / np.pi
    LWout = nans.copy()
    SWout = nans.copy()
    rh_data = nans.copy()
    NR = nans.copy()
    SWin = SWin[0]
    LWin = LWin[0]
    tcc = tcc[0]
    ntimesteps = len(data_hours)
elif climate_input in ['AWS']:
    aws = class_climate.AWS(eb_prms.AWS_fn,dates_table)
    temp_data = aws.temp
    tp_data = aws.tp
    dtemp_data = aws.dtemp
    rh_data = aws.rh
    SWin = aws.SWin
    SWout = aws.SWout
    LWin = aws.LWin
    LWout = aws.LWout
    NR = aws.NR
    wind = aws.wind
    winddir = aws.winddir
    tcc = aws.tcc
    sp_data = aws.sp
    elev_data = aws.elev
    depBC = nans.copy()
    depdust = nans.copy()
    ntimesteps = len(temp_data)

1 glaciers in region 1 are included in this model run: ['00570']
This study is focusing on 1 glaciers in region [1]


In [9]:
#initialize variables to be adjusted
temp = np.zeros((n_bins,ntimesteps))
tp = np.zeros((n_bins,ntimesteps))
sp = np.zeros((n_bins,ntimesteps))
dtemp = np.zeros((n_bins,ntimesteps))

# define function to calculate vapor pressure (needed for RH)
e_func = lambda T_C: 610.94*np.exp(17.625*T_C/(T_C+243.04))  #vapor pressure in Pa, T in Celsius
#loop through each elevation bin and adjust climate variables by lapse rate/barometric law
for idx,z in enumerate(eb_prms.bin_elev):
    # correct temperature according to lapserate
    temp[idx,:] = temp_data + eb_prms.lapserate*(z-elev_data)
    # correct precipitation according to lapserate, precipitation factor
    if len(np.array(eb_prms.kp).flatten()) > 1:
        tp[idx,:] = tp_data*(1+eb_prms.precgrad*(z-elev_data))*eb_prms.kp[idx]
    else:
        tp[idx,:] = tp_data*(1+eb_prms.precgrad*(z-elev_data)) # *eb_prms.kp for GCM******
    # correct surface pressure according to barometric law
    sp[idx,:] = sp_data*np.power((temp_data + eb_prms.lapserate*(z-elev_data)+273.15)/(temp_data+273.15),
                        -eb_prms.gravity*eb_prms.molarmass_air/(eb_prms.R_gas*eb_prms.lapserate))
    # if RH is not empty, get dtemp data from it
    if not np.all(np.isnan(rh_data)): 
        dtemp_data = rh_data / 100 * e_func(temp[idx,:])
        dtemp[idx,:] = dtemp_data + eb_prms.lapserate_dew*(z-elev_data)-273.15
        rh = np.array([rh_data]*n_bins)
    else:
        rh = e_func(dtemp) / e_func(temp) * 100

dates = pd.date_range(startdate,enddate,freq='h')

# ===== SET UP CLIMATE DATASET =====
bin_idx = np.arange(0,n_bins)
climateds = xr.Dataset(data_vars = dict(
    bin_elev = (['bin'],eb_prms.bin_elev,{'units':'m'}),
    SWin = (['time'],SWin,{'units':'J m-2'}),
    SWout = (['time'],SWout,{'units':'J m-2'}),
    LWin = (['time'],LWin,{'units':'J m-2'}),
    LWout = (['time'],LWout,{'units':'J m-2'}),
    NR = (['time'],NR,{'units':'J m-2'}),
    tcc = (['time'],tcc,{'units':'0-1'}),
    wind = (['time'],wind,{'units':'m s-1'}),
    winddir = (['time'],winddir,{'units':'deg'}),
    depBC = (['time'],depBC,{'units':'kg s-1'}),
    depdust = (['time'],depdust,{'units':'kg s-1'}),),
    coords = dict(
        bin=(['bin'],bin_idx),
        time=(['time'],dates)
        ))

climateds = climateds.assign(bin_temp = (['bin','time'],temp,{'units':'C'}))
climateds = climateds.assign(bin_tp = (['bin','time'],tp,{'units':'m'}))
climateds = climateds.assign(bin_sp = (['bin','time'],sp,{'units':'Pa'}))
climateds = climateds.assign(bin_rh = (['bin','time'],rh,{'units':'%'}))
